<a href="https://colab.research.google.com/github/hesandism/ML-projects/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2026-01-09 16:14:30--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M   148MB/s    in 0.2s    

2026-01-09 16:14:30 (148 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [46]:
user_counts= df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index
df_ratings_filtered_by_users = df_ratings[df_ratings['user'].isin(active_users)]

book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index
df_ratings_filtered_by_users_books = df_ratings_filtered_by_users[df_ratings_filtered_by_users['isbn'].isin(popular_books)]
df_combined = df_ratings_filtered_by_users_books.merge(df_books[['isbn','title']], on='isbn')


In [47]:
book_user_matrix = df_combined.pivot_table(index='title',columns='user',values='rating').fillna(0)
book_user_sparse = csr_matrix(book_user_matrix.values)

In [48]:
N_NEIGHBORS = 6
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=N_NEIGHBORS)
model_knn.fit(book_user_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [54]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    global model_knn
    global book_user_matrix

    if book not in book_user_matrix.index:
          return [book, []]
    recommended_books = []
    recommended_books.append(book)
    neighbors_list = []
    reqired_book_2d = book_user_matrix.loc[book].to_numpy().reshape(1,-1)

    distance , indices = model_knn.kneighbors(X=reqired_book_2d,n_neighbors=N_NEIGHBORS)
    distance[0] = distance[0][::-1]
    indices[0] = indices[0][::-1]
    for i in range(N_NEIGHBORS-1):
        d = distance[0][i].item()
        idx = indices[0][i]
        neighbors_list.append([book_user_matrix.index[idx],d])
    recommended_books.append(neighbors_list)
    return recommended_books


In [56]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    print("diff    :",abs(recommends[1][i][1] - recommended_books_dist[i]),recommends[1][i][0])
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
diff    : 0.0016210794448852095 I'll Be Seeing You
diff    : 0.0008583474159240545 The Weight of Water
You passed the challenge! 🎉🎉🎉🎉🎉
